## TODO

- [ ] bindings 'enter' in list -> inspect; 'enter' in insect -> list
- [ ] after inspect, rename, edit, complete,... show list but with original tracker selected and highlighted - wherever it is now [id to (page, row)]
- [ ] workout feedback while entering date
- [ ] maybe change edit to select one from history to edit or delete

mode_hash = {
    mode1: {key1: method1, key2: method2, ...},
    mode2: {key3: method3, key4: method4, ...},
}

Keys in a mode are bound to the corresponding method with the condition that the active mode is that mode. 


In [ ]:
selected =
showing = False
has_selection = showing or selected


class Dialog:
    """
    Keeps track of the current mode and the selected_id. Each method in mode2bindings is bound to the corresponding key with the condition that the mode is the current mode.
    """
    filters = [menu_mode, select_mode, bool_mode, integer_mode, character_mode, dialog_visible, message_visible, input_visible]

    # TODO: maybe have Tracker methods for complete, ...

    mode2bindings = {
        'main': {
            'f1': menu,
            'f2': about,
            'f3': maybe_update,
            'f4': settings,
            # ...
            'f8': help,
            '^q': quit,
            'S': sort,
            'T': select_tag, # FIXME
            'enter', toggle_details, # show details
            'N': new,
            'C': complete,
            'R': rename,
            'H': history,
            'D': delete,
            '123': select_page,
            'abc': select_tag,
            },
        'new': {
           'enter': handle_new,
           'escape': cancel_new,
            },
        'complete' : {
            'enter': handle_complete,
            'escape': cancel_complete,
            },
        'rename' : {
            'enter': handle_rename,
            'escape': cancel_rename,
            },
        'history' : {
            'enter': handle_history,
            'escape': cancel_history,
            },
        }

    def new(self):
        self.set_mode('new') # set keybindings, ...
        self.message_control.text = """\
 Enter the name of the new tracker. Optionally append a comma and the datetime of the first completion, and again, optionally, another comma and the timedelta of the expected interval until the next completion, e.g. 'name, 3p wed, +7d'.  Press 'enter' to save changes or '^c' to cancel.
"""
            self.app.layout.focus(input_area)

    def __init__(self) -> None:
        self.mode = 'main'
        self.selected_id = None
        self.showing_details = False
        self.set_bindings()

    def set_bindings(self):
        self.kb = KeyBindings()
        for mode, bindings in self.mode2bindings.items():
            for key, method in bindings.items():
                self.kb.add(key, filter=Condition(lambda: self.mode == mode))(method)

        tag_keys = list(string.ascii_lowercase)
        for key in tag_keys:
            kb.add(key, filter=Condition(lambda: self.mode == 'main'), eager=True)(lambda event, key=key: self.get_tracker_from_tag(event, key))
        page_keys = list(range(1, 10))
        for key in tag_keys:
            kb.add(key, filter=Condition(lambda: self.mode == 'main'), eager=True)(lambda event, key=key: self.get_tracker_from_tag(event, key))






In [ ]:
# Example methods
def method1(arg1, arg2):
    print(f"Method 1 called with {arg1} and {arg2}")

def method2(arg1, arg2):
    print(f"Method 2 called with {arg1} and {arg2}")

# Dictionary with methods as keys
method_dict = {
    method1: "This is method 1",
    method2: "This is method 2"
}

# Access the value for each method
print(method_dict[method1])  # Outputs: This is method 1
print(method_dict[method2])  # Outputs: This is method 2

# You can still run the methods normally
methods = [method1, method2]
for method in methods:
    method('arg1_value', 'arg2_value')


In [ ]:
def run_methods(methods_with_args):
    for method, args in methods_with_args.items():
        method(*args)  # Execute the method with the associated arguments

# Define methods and their corresponding arguments in a dictionary
methods_with_args = {
    method1: ('arg1_value', 'arg2_value'),
    method2: ('another_arg1', 'another_arg2'),
}

# Run methods in a loop
run_methods(methods_with_args)


## node to node transitions
(beg_node, (keypress, method), end_node), ...

Nodes:
- main (list view, cursor on row 0 - no selection)
- select (list view and tag_keys active)
- selected (list view, tracker_selected and highlighted)

E.g., 
- (viewing_main, [F8, do_help], viewing_help)
- (main)
- (main, [t, get_selection], select), (select, [tag_key, select_tag], selected)
- (main or selected, [cursor_key, move_selection], selected or main)
- (main, [n, start_new], new_entry), (new_entry, [escape, cancel_entry], main), (new_entry, [return, record_new], selected) 

transitions = [main (menu/list/select), select_to_selected, selected_to_inspect, ..., selected_to_delete, ... inspect_to_rename, ..., inspect_to_list (change selection)

each transition starting point has an associated set of keybindings which effectively serve as the menu for possible end points for transitions from the starting point. E.g, menu binds F8 and various sundry menu keys so that pressing F8 in menu mode transitions to displaying help - so menu_help is the transition associated with F8.

In [ ]:
nodes that require selection = ['complete', 'delete', 'rename', 'edit', 'inspect']
other_nodes = ['main', 'select', 'new', 'sort', 'selected']
all_nodes = select_nodes + other_nodes




def list():
    # set the keybindings filter
    set_mode('menu')
    # stuff to display list of trackers
    pass

def select():
    # set the keybindings filter
    tracker = get_tracer_from_row()
    if tracker:
        return tracker

    set_mode('select')
    display_message('Select a tracker by pressing the key corresponding to its tag')

    def _(event):
        key_pressed = event.key_sequence[0].key
    # stuff to select tracker and set self.tracker to the result
    pass


- main
    - function keys
    - home/space -> list row 0, page 0
    - sort
    - 't', cursor_keys -> select row
    - needs selection (prompt for and get selection if a tracker is not already selected)
        - 'c'



# methods beginning from 'main' for various tasks
# each method returns the next method, if any


steps = {
    'main': [menu_keys, cursor_keys],
    'select': [tag_keys, 'escape'],
    'selected': [menu_keys, 'complete', 'delete', 'rename', 'edit', 'inspect', 'new', 'sort'],
    'complete': [select, edit_completion],
    'delete': [select, confirm_deletion],
    'rename': [select, edit_name],
    'edit': [select, edit_history],
    'inspect': [select, inspect],
    'new': [edit_name],
    'sort': [],
}



class Task:
    # preserve state for task name, relevant id and node
    def __init__(self, task: str = 'start'):
        self.reset(task=task)

    def reset(self, task: str = 'start'):
        self.steps = [x for x in steps.get(task, [])]
        set_mode('menu')
        self.node = self.steps.pop(0)
        self.tracker = None



active_task = Task()

def start_task(task: str):
    active_task.reset(task=task)
    if active_task.node == 'select':
        tracker = get_tracker_from_row()
        if not tracker:
            active_task.update(node='select') # sets tag select key bindings
            tracker = select_tracker()
        if tracker:
            active_task.steps.pop(0)
        else:
            active_task.reset()
            return

    while active_task.steps:
        active_task.node = active_task.steps.pop(0)
        # this sets the keybindings filter for this node
        if active_task.node










In [2]:
def run_methods(methods, *args):
    for method in methods:
        method(*args)  # Execute each method with the provided arguments

# Example methods
def method1(arg1, arg2):
    print(f"Method 1 called with {arg1} and {arg2}")

def method2(arg1, arg2):
    print(f"Method 2 called with {arg1} and {arg2}")

# List of method references
methods = [method1, method2]

# Run all methods in a loop
run_methods(methods, 'arg1_value', 'arg2_value')


Method 1 called with arg1_value and arg2_value
Method 2 called with arg1_value and arg2_value


# Dialogs

## Strategy

Components each of which does one thing well and which play nice with one another.

### Interfaces

- Conditional message area
    - input only keypress, tag, y/n, ...

- Conditional dialog
    - message/response window
    - input window (strings)

- Display area (non interactive, list view, inspect view, ...)

## Uses

### new

### complete

### edit (history of completions)

### delete

### rename

### inspect

### sort

### settings


## Components

### get character
- get tag (get tracker)
- get yes/no
- sort

### get tracker (from row or tag)
- complete
- edit
- delete
- rename
- inspect

### get string input with feedback
- new
- edit
- complete
- rename
- settings

## Key Bindings

### Common
- c-q (quit)
- escape (cancel current operation)

### menu 
- menu keys (including function keys)

### tags
- char for tags in active page


get_tracker():
    # called if needs_selection == True
    self.selected_tracker = None
    if tracker from row:
        set selected_tracker = tracker
        return True
    set_mode('selection')
    if key_press == 'escape':
        return False
    tracker = get(tracker_from_tag.get(activepage, keypress), None)
    if tracker:
        set selected_tracker = tracker
        return True
    return False



needs_selection?
steps = [action]
for action in steps:
    set_mode(action)
    ok, result = handle(action)
    if not ok:
        display(result)
        return
    
Dialog(menu_key_pressed: str)
    action = dict: key_pressed -> [steps]
    steps
        (test: bool, if_true_action, if_false_action)
        (needs_selection?, get_tracker, None)
        (key_binding: KeyBinding = None, dialog: list[message, input])
    needed = list of tuples (tracker:bool, )

   action? complete, rename, new, ...

Each step 


get_tracker_from_row_or_tag logic
    if tracker from row:
        return tracker
    set_mode(tag)
    message('select tag')
    wait for tag or escape press
    if tracker from tag
        return tracker
    return None


rename logic
menu_keys
    'r' pressed
    if selection_needed('r')
        tracker = get_tracker_from_row_or_tag
        if not tracker:
            return
    dispatch('r')
        do_rename


In [4]:
from datetime import datetime, date, timedelta

def make_screenshot_examples():
    now = datetime.now().replace(microsecond=0,second=0,minute=0,hour=0)
    # for histories we need intervals 1 and 2 that
    # |i1 - (i1 + i2)/2| = |i2 - i1|/2
    # |i2 - (i1 + i2)/2| = |i1 - i2|/2
    # mad = (|i2 - i1|/2)*2/2 = |i2 - i1|/2
    # => mad = 1d2h => i2-i1
    # avg = (i1 + i2)/2 = 10d  = i2 - i1 = 20d - 2*i1 = 1d2h
    # 2*i1 = 20d - 1d2h = 18d22h => i1 = 9d11h
    # => i2 = 1d2h + 9d11h = 10d13h
    i1 = timedelta(days=9, hours=11)
    i2 = timedelta(days=10, hours=13)
    latef = now - timedelta(days=13, hours=5)
    late1 = latef - i1 - i2
    late2 = latef - i2
    duef = now - timedelta(days=9,hours=8)
    due1 = duef - i1 - i2
    due2 = duef - i2
    earlyf = now - timedelta(days=3, hours=6)
    early1 = earlyf - i1 - i2
    early2 = earlyf - i2
    onlyone1 = now - i2


    egs = [
        (1001, 'late', [late1, late2, latef]),
        (1002, 'due', [due1, due2, duef]),
        (1003, 'early', [early1, early2, earlyf]),
        (1004, 'only one', [onlyone1]),
        (1005, 'none', [])
    ]

    for eg in egs:
        dates = "; ".join([d.strftime("%Y-%m-%d %H:%M") for d in eg[2]])
        print(eg[0], eg[1], dates)

make_screenshot_examples()

1001 late 2024-08-20 19:00; 2024-08-30 06:00; 2024-09-09 19:00
1002 due 2024-08-24 16:00; 2024-09-03 03:00; 2024-09-13 16:00
1003 early 2024-08-30 18:00; 2024-09-09 05:00; 2024-09-19 18:00
1004 only one 2024-09-12 11:00
1005 none 
